**Homework main task**: overcome Mistral hallucinations in chat mode.

### Plan:

- Test the pre-trained model as an online store assistant.
- Completely train the model using the LoRA method on data from chatbot communication with clients on various products.
- Test the completed model.

:You need to fill in all the blankes in the code (there are 7 spaces in total).

**Note!** Please safe all cells outputs, otherwise the task will not be accepted!

# Fine-tune Mistral 7B.

In [1]:
# !pip install peft transformers bitsandbytes accelerate trl datasets -U -qqq

In [2]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

Let's first test how the Mistral 7b, pre-trained in Russian (the original model was trained only in English), will cope with this task. Let the model play the role of an assistant for an online smartphone store.

In [3]:
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
RESPONSE_TEMPLATE = "<s>bot\n"
SYSTEM_PROMPT = """Ты – чат-бот технической поддержки Xiaomi Store, который помогает клиенту выбрать наиболее подходящий для него смартфон. Опираясь на описание смартфонов, помоги клиенту выбрать наиболее подходящий для него смартфон. Если ответа на вопрос клиента нет в приведенном описании, ответь "У меня не достаточно информации для ответа на ваш вопрос. Обратитесь пожалуйста к менеджеру в telegram".
Описание смартфонов из интернет-магазина сотовой связи Xiaomi Store:
1. Смартфон Xiaomi Redmi Note 10 Pro: Этот смартфон оснащен дисплеем Super AMOLED с разрешением 1080 x 2400 пикселей, что обеспечивает четкую и яркую картинку. Он также имеет камеру на 64 Мп с возможностью записи видео в 4K. Процессор Snapdragon 732G обеспечивает отличную производительность, а аккумулятор на 5020 мАч обеспечивает долгое время работы.
2. Смартфон Xiaomi Mi 11 Lite: Этот смартфон имеет ультратонкий и легкий дизайн, который удобно держать в руке. Он оснащен 6.55-дюймовым AMOLED-дисплеем с разрешением 1080 x 2400 пикселей. Камера на 64 Мп способна делать качественные фотографии, а быстрый процессор Qualcomm Snapdragon 732G позволяет работать с приложениями плавно.
3. Смартфон Xiaomi Redmi 9T: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 1080 x 2340 пикселей, который обеспечивает реалистичную цветопередачу. Камера на 48 Мп делает четкие фото, а процессор Snapdragon 662 обеспечивает быструю работу. Аккумулятор на 6000 мАч позволяет использовать устройство долгое время без подзарядки.
4. Смартфон Xiaomi Mi 10T Pro: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Он имеет камеру на 108 Мп, что позволяет делать потрясающие фотографии. Процессор Qualcomm Snapdragon 865 обеспечивает высокую производительность, а аккумулятор на 5000 мАч гарантирует долгое время работы.
5. Смартфон Xiaomi Redmi Note 9 Pro: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Камера на 64 Мп с поддержкой искусственного интеллекта позволяет делать яркие и четкие фотографии. Процессор Snapdragon 720G обеспечивает отличную производительность, а аккумулятор на 5020 мАч обеспечивает долгое время работы.
6. Смартфон Xiaomi Mi 10T Lite: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Он имеет камеру на 64 Мп, что позволяет делать яркие и детальные фотографии. Процессор Qualcomm Snapdragon 750G ускоряет работу с приложениями, а аккумулятор на 4820 мАч обеспечивает долгую автономность.
7. Смартфон Xiaomi Redmi Note 8 Pro: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 1080 x 2340 пикселей. Камера на 64 Мп делает четкие и яркие фото, а процессор MediaTek Helio G90T обеспечивает плавную работу. Аккумулятор на 4500 мАч достаточно емкий для долгого использования.
8. Смартфон Xiaomi Mi 10 Pro: Этот смартфон оснащен 6.67-дюймовым AMOLED-дисплеем с разрешением 1080 x 2340 пикселей. Камера на 108 Мп и оптическая стабилизация изображения позволяют делать высококачественные фотографии. Процессор Snapdragon 865 ускоряет работу с приложениями, а аккумулятор на 4500 мАч достаточно емкий.
9. Смартфон Xiaomi Redmi 9C: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 720 x 1600 пикселей. Он имеет камеру на 13 Мп, которая делает четкие фото в хороших условиях освещения. Процессор MediaTek Helio G35 обеспечивает достаточную производительность для базовых задач, а аккумулятор на 5000 мАч обеспечивает долгое время работы.
10. Смартфон Xiaomi Mi 11 Ultra: Этот смартфон оснащен 6.81-дюймовым AMOLED-дисплеем с разрешением 1440 x 3200 пикселей. Он имеет камеру на 50 Мп, а также вспомогательные камеры для различных эффектов съемки. Процессор Snapdragon 888 обеспечивает высокую производительность, а аккумулятор на 5000 мАч гарантирует долгое время работы."""

In [4]:
class Conversation:
    def __init__(
        self,
        message_template=MESSAGE_TEMPLATE,
        system_prompt=SYSTEM_PROMPT,
        response_template=RESPONSE_TEMPLATE
    ):
        # Инициализация класса Conversation с шаблонами сообщений и системным запросом.
        # message_template - шаблон для каждого сообщения (например, форматирование текста).
        # system_prompt - начальный запрос, задающий контекст разговора.
        # response_template - шаблон для ответа, который будет добавлен в конце текста.

        self.message_template = message_template  # Сохранение шаблона сообщений.
        self.response_template = response_template  # Сохранение шаблона ответа.
        # Инициализация списка сообщений с первым сообщением от системы, определяющим контекст беседы.
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        # Метод для добавления сообщения пользователя в список сообщений.
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        # Метод для добавления сообщения бота в список сообщений.
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        # Метод для формирования финального текста запроса, который будет отправлен для генерации ответа.
        final_text = ""  # Инициализация переменной для хранения окончательного текста.
        for message in self.messages:
            # Форматирование каждого сообщения в соответствии с message_template.
            message_text = self.message_template.format(**message)
            final_text += message_text  # Добавление отформатированного текста к итоговому.

        final_text += RESPONSE_TEMPLATE  # Добавление шаблона ответа в конце текста.
        return final_text.strip()  # Возвращение итогового текста без лишних пробелов в начале и в конце.


In [5]:
def generate(model, tokenizer, prompt, generation_config):
    # Функция для генерации текста на основе заданного промпта с использованием модели и токенизатора.

    # Токенизация промпта: превращаем текстовый промпт в числовые представления (токены),
    # которые понимает модель. Параметр return_tensors="pt" означает, что данные
    # будут возвращены в формате PyTorch тензоров. add_special_tokens=False указывает,
    # что специальные токены (например, [CLS], [SEP]) не будут добавлены.
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)

    # Переносим данные на устройство, на котором работает модель (например, GPU или CPU).
    data = {k: v.to(model.device) for k, v in data.items()}

    # Генерация выходных идентификаторов (токенов) с использованием модели.
    # Параметр generation_config задает конфигурацию генерации, например,
    # максимальную длину ответа, температуру и т.д.
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]

    # Обрезаем выходные идентификаторы, чтобы исключить исходные токены промпта.
    # Таким образом, оставляем только сгенерированный моделью ответ.
    output_ids = output_ids[len(data["input_ids"][0]):]

    # Декодируем токены обратно в текст, исключая специальные токены, такие как [PAD] или [SEP].
    output = tokenizer.decode(output_ids, skip_special_tokens=True)

    # Возвращаем итоговый сгенерированный текст, убирая лишние пробелы в начале и в конце.
    return output.strip()

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}

In [7]:
# Загружаем конфигурацию модели PeftConfig из предобученной модели, используя ее имя.
config = PeftConfig.from_pretrained(MODEL_NAME)

# Загружаем предобученную модель для задачи автозавершения текста (Causal Language Modeling).
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="./offload",
    low_cpu_mem_usage=True
)

model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    offload_folder="./offload",
    low_cpu_mem_usage=True
)


# Переводим модель в режим оценки, чтобы отключить функции, используемые только при обучении,
# такие как dropout, и настроить модель на генерацию предсказаний.
model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Some parameters are on the meta device because they were offloaded to the cpu.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [8]:
conversation = Conversation()

In [9]:
while True:
    user_uttr = input("Клиент: ")
    conversation.add_user_message(user_uttr)
    prompt = conversation.get_prompt(tokenizer)
    output = generate(model, tokenizer, prompt, generation_config)
    output = output.split("bot")[0].strip()
    print(f"Чат-бот: {output}")

Чат-бот: Да, конечно, могу помочь! Планшеты, которые поддерживают 4К видео, обычно имеют большой экран и высокую разрешающую способность. Однако, стоит учесть, что большинство планшетов не предназначены специально для просмотра 4К видео, так как это требует большой мощности и большого объема памяти.

Вот несколько планшетов, которые могут поддерживать 4К видео:

1. Apple iPad Pro (2020): Этот планшет имеет большой экран 12.9 дюймов с высоким разрешением 2732 x 2048 пикселей. Он оснащен процессором Apple M1, который обеспечивает высокую производительность.

2. Samsung Galaxy Tab S7+: Этот планшет имеет экран 12.4 дюймов с разрешением 2800 x 1752 пикселей. Он оснащен процессором Qualcomm Snapdragon 865+, который обеспечивает высокую производительность, и поддерживает 4К видео.

3. Microsoft Surface Pro 7: Этот планшет имеет экран 2736 x 1824 пикселей и поддерживает 4К видео. Он оснащен процессором Intel Core i5 или i7, который обеспечивает высокую производительность и поддерживает 4К вид

KeyboardInterrupt: 

Let's start a conversation. Try asking to tell about tablets that have 4K video quality. We see that the bot starts to invent non-existent information, that is, it starts to hallucinate.

In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

Downloading data for additional fine-tuning.

In [2]:
# !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1_xKwkNFisKzDQ1CPPDqvKWdPRzZvO4MT' -O company_cases.json

In [3]:
import json
import torch
from datasets import Dataset
from peft import LoraConfig
from sklearn.model_selection import train_test_split
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [4]:
with open("company_cases.json", 'r') as inp:
    raw_dataset = json.load(inp)

train_raw_dataset, test_raw_dataset = train_test_split(raw_dataset)
train_test_raw_dataset = {"train": train_raw_dataset, "test": test_raw_dataset}

Let's convert the dataset into the format used in the Dataset class.

In [5]:
raw_dataset_dict = {}
for data_type in ["train", "test"]:
    raw_dataset_dict[data_type] = {
        "instruction": [element['instruction'] for element in train_test_raw_dataset[data_type]],
        "personality": [element['personality'] for element in train_test_raw_dataset[data_type]],
        "context": [element['context'] for element in train_test_raw_dataset[data_type]],
        "dialog_start_line": [element['instruction'] for element in train_test_raw_dataset[data_type]],
        "dialog": [element['dialog'] for element in train_test_raw_dataset[data_type]]
    }

train_dataset = Dataset.from_dict(raw_dataset_dict["train"])
test_dataset = Dataset.from_dict(raw_dataset_dict["test"])

Function for formatting the prompt.

In [6]:
def formatting_prompt_func(example):
    prompt = f"<s>system\n{example['instruction']}"
    if example['personality']:
        prompt += f"\n{example['personality']}"
    prompt += f"\n{example['context']}"
    if example['dialog_start_line']:
        prompt += f"\n{example['dialog_start_line']}"
    prompt += "</s>"
    for element in example['dialog']:
        prompt += f"<s>{element['role']}\n{element['content']}</s>"
    return prompt

For additional training, we will take not the ready Russian-language Mistral, but the pre-trained Mistral on the Open-Orca dataset.

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/Mistral-7B-OpenOrca")
tokenizer.pad_token_id = 0

Let's make the gradient during training flow only through the tokens of the last replica of the chatbot, and let's make the labels for the remaining tokens equal to -100.

In [8]:
response_template = "bot\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [9]:
# Будем обучать модель в int4 для уменьшения требуемой видеопамяти

model = AutoModelForCausalLM.from_pretrained(
    "Open-Orca/Mistral-7B-OpenOrca",
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True
    )
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Initializing LoraConfig.

In [10]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM"
)

Setting parameters for additional training.

In [12]:
training_arguments = TrainingArguments(
    output_dir="checkpoints",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=2e-4,
    # fp16=True,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True
)

Disable tokenizer parallelism to avoid deadlocks.

In [13]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Starting fine-tuning.

In [14]:
trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    formatting_func=formatting_prompt_func,
    data_collator=collator,
    peft_config=peft_config,
    args=training_arguments,
    # max_seq_length=2048
)

trainer.train()

Applying formatting function to train dataset:   0%|          | 0/8354 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/8354 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/8354 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/8354 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/8354 [00:00<?, ? examples/s]

...

Applying chat template to eval dataset:   0%|          | 0/2785 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/2785 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/2785 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/home/gotheartem/Projects/ioai-hw/.venv/lib/python3.11/site-packages/trl/trainer/utils.py:145: UserWarning: Could not find response key `bot
` in the following instance: <s><s> system
Ниже представлено описание 10 инвестиционных предложений банка Сбербанк с указанием комиссии, доходности и размера минимального взноса. Ты – чат-бот технической поддержки сбербанк, который помогает клиенту выбрать наиболее подходящее для него инвестиционное предложение. Опираясь на описание инвестиционных предложений, помоги клиенту выбрать наиболее подходящее для него инвестиционное предложение. Пользователь может задавать вопросы про инвестиционное предложение, которого нет в таблице, в этом случае чат-бот говорит, что данного инвестиционного предлож

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 11.72 GiB of which 168.88 MiB is free. Including non-PyTorch memory, this process has 10.65 GiB memory in use. Of the allocated memory 10.24 GiB is allocated by PyTorch, and 234.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Test the resulting model.

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

Loading the adapter checkpoint.

In [ ]:
BASE_MODEL = "Open-Orca/Mistral-7B-OpenOrca"
ADAPTER_MODEL = "checkpoints/checkpoint-1044"
MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
RESPONSE_TEMPLATE = "<s>bot\n"
SYSTEM_PROMPT = """Ты – чат-бот технической поддержки Xiaomi Store, который помогает клиенту выбрать наиболее подходящий для него смартфон. Опираясь на описание смартфонов, помоги клиенту выбрать наиболее подходящий для него смартфон. Если ответа на вопрос клиента нет в приведенном описании, ответь "У меня не достаточно информации для ответа на ваш вопрос. Обратитесь пожалуйста к менеджеру в telegram".{personality}
Описание смартфонов из интернет-магазина сотовой связи Xiaomi Store:
1. Смартфон Xiaomi Redmi Note 10 Pro: Этот смартфон оснащен дисплеем Super AMOLED с разрешением 1080 x 2400 пикселей, что обеспечивает четкую и яркую картинку. Он также имеет камеру на 64 Мп с возможностью записи видео в 4K. Процессор Snapdragon 732G обеспечивает отличную производительность, а аккумулятор на 5020 мАч обеспечивает долгое время работы.
2. Смартфон Xiaomi Mi 11 Lite: Этот смартфон имеет ультратонкий и легкий дизайн, который удобно держать в руке. Он оснащен 6.55-дюймовым AMOLED-дисплеем с разрешением 1080 x 2400 пикселей. Камера на 64 Мп способна делать качественные фотографии, а быстрый процессор Qualcomm Snapdragon 732G позволяет работать с приложениями плавно.
3. Смартфон Xiaomi Redmi 9T: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 1080 x 2340 пикселей, который обеспечивает реалистичную цветопередачу. Камера на 48 Мп делает четкие фото, а процессор Snapdragon 662 обеспечивает быструю работу. Аккумулятор на 6000 мАч позволяет использовать устройство долгое время без подзарядки.
4. Смартфон Xiaomi Mi 10T Pro: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Он имеет камеру на 108 Мп, что позволяет делать потрясающие фотографии. Процессор Qualcomm Snapdragon 865 обеспечивает высокую производительность, а аккумулятор на 5000 мАч гарантирует долгое время работы.
5. Смартфон Xiaomi Redmi Note 9 Pro: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Камера на 64 Мп с поддержкой искусственного интеллекта позволяет делать яркие и четкие фотографии. Процессор Snapdragon 720G обеспечивает отличную производительность, а аккумулятор на 5020 мАч обеспечивает долгое время работы.
6. Смартфон Xiaomi Mi 10T Lite: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Он имеет камеру на 64 Мп, что позволяет делать яркие и детальные фотографии. Процессор Qualcomm Snapdragon 750G ускоряет работу с приложениями, а аккумулятор на 4820 мАч обеспечивает долгую автономность.
7. Смартфон Xiaomi Redmi Note 8 Pro: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 1080 x 2340 пикселей. Камера на 64 Мп делает четкие и яркие фото, а процессор MediaTek Helio G90T обеспечивает плавную работу. Аккумулятор на 4500 мАч достаточно емкий для долгого использования.
8. Смартфон Xiaomi Mi 10 Pro: Этот смартфон оснащен 6.67-дюймовым AMOLED-дисплеем с разрешением 1080 x 2340 пикселей. Камера на 108 Мп и оптическая стабилизация изображения позволяют делать высококачественные фотографии. Процессор Snapdragon 865 ускоряет работу с приложениями, а аккумулятор на 4500 мАч достаточно емкий.
9. Смартфон Xiaomi Redmi 9C: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 720 x 1600 пикселей. Он имеет камеру на 13 Мп, которая делает четкие фото в хороших условиях освещения. Процессор MediaTek Helio G35 обеспечивает достаточную производительность для базовых задач, а аккумулятор на 5000 мАч обеспечивает долгое время работы.
10. Смартфон Xiaomi Mi 11 Ultra: Этот смартфон оснащен 6.81-дюймовым AMOLED-дисплеем с разрешением 1440 x 3200 пикселей. Он имеет камеру на 50 Мп, а также вспомогательные камеры для различных эффектов съемки. Процессор Snapdragon 888 обеспечивает высокую производительность, а аккумулятор на 5000 мАч гарантирует долгое время работы."""

In [ ]:
class Conversation:
    def __init__(
        self,
        message_template=MESSAGE_TEMPLATE,
        system_prompt=SYSTEM_PROMPT,
        response_template=RESPONSE_TEMPLATE,
        personality=None
    ):
        self.message_template = message_template
        self.response_template = response_template
        if personality:
            system_prompt = system_prompt.format(personality=personality)
        else:
            system_prompt = system_prompt.format(personality="")
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += RESPONSE_TEMPLATE
        return final_text.strip()

In [ ]:
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False)
tokenizer.pad_token_id = 0
generation_config = GenerationConfig(
    pad_token_id=0,
    bos_token_id=1,
    eos_token_id=2,
    temperature=0.2,
    top_p=0.9,
    # Ваш код здесь
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, torch_dtype=torch.float16)
model.eval()

In [ ]:
conversation = Conversation()

We start the dialogue again and ask about the tablets. Were you able to overcome the hallucinations?

In [ ]:
while True:
    user_uttr = input("Клиент: ")
    print(f"Клиент: {user_uttr}")
    conversation.add_user_message(user_uttr)
    prompt = conversation.get_prompt(tokenizer)
    output = generate(model, tokenizer, prompt, generation_config)
    output = output.split("bot")[0].replace("<s>", "").replace("</s>", "").strip()
    print(f"Чат-бот: {output}")